# Explainable Quantum Clustering Method to Model Medical Data

1. Importing machine learning and quantum machine learning libraries
2. dataset preparation: Assume that the dataset has been downloaded as a csv file onto a local computer,the dataset file can then be loaded into a pandas DataFrame.
3. loading, preprocessing data
4. Initally define the number of clusters(cluster center is Centroid), datapoints, and features
5. Calculate distance from datapoint to centroids
6. Updating centroids (Store old centroid to calculate new centroid)
7. Assign the datapoint to a closest centroid.
8. Calculate silhouette score for each datapoint(it should be positive value between 0 to 1)
9. Calculate accuracy to show the cluster quality
10. Install LIME : Provide explanations for individual predictions as a solution to determining trust.
11. Pipeline is created for conveninence
12. Tabular explainer is selected
13. Explaining model predictions


In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from numpy import pi
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit
from qiskit import Aer, execute

#qk-means Algorithm
# Number of clusters
k = 2
# Number of training data
n = data.shape[0]
# Number of features in the data
c = data.shape[1]

# Generate random centers
mean = np.mean(data, axis = 0)
std = np.std(data, axis = 0)
centers = np.random.randn(k,c)*std + mean

# Static data to test
centers = np.array([[1, 2,3,4,2,2,3], [6,5,6,7,6,5,6]])
print(centers)

# Plot the data and the centers generated as random
colors=['green', 'blue']
for i in range(n):
    plt.scatter(data[i, 0], data[i,1], s=7, color = colors[int(category[i])])
plt.scatter(centers[:,0], centers[:,1], marker='*', c='g', s=150)

def point_centroid_distances(point, centroids):
    
    # Calculating theta and phi values
    phi_list = [((x + 1) * pi / 2) for x in [point[0], centroids[0][0], centroids[1][0], centroids[2][0]]]
    theta_list = [((x + 1) * pi / 2) for x in [point[1], centroids[0][1], centroids[1][1], centroids[2][1]]]

    # Create a 3 qubit QuantumRegister - three for the vectors, and 
    # two for the ancillary qubit
    qreg = QuantumRegister(3, 'qreg')

    # Create a one bit ClassicalRegister to hold the result
    # of the measurements
    creg = ClassicalRegister(2, 'creg')

    qc = QuantumCircuit(qreg, creg, name='qc')

    # Get backend using the Aer provider
    backend = Aer.get_backend('qasm_simulator')

    # Create list to hold the results
    results_list = []

    # Estimating distances from the new point to the centroids
    for i in range(1, 4):
        # Apply a Hadamard to the ancillary
        qc.h(qreg[2])

        # Encode new point and centroid
        qc.u3(theta_list[0], phi_list[0], 0, qreg[0])           
        qc.u3(theta_list[i], phi_list[i], 0, qreg[1]) 

        # Perform controlled swap
        qc.cswap(qreg[2], qreg[0], qreg[1])
        # Apply second Hadamard to ancillary
        qc.h(qreg[2])

        # Measure ancillary
        qc.measure(qreg[2], creg[0])

        # Reset qubits
        qc.reset(qreg)

        # Register and execute job
        job = execute(qc, backend=backend, shots=5000)
        result = job.result().get_counts(qc)
        try:
            results_list.append(result['1'])
        except:
            results_list.append(0)


    return results_list

centers_old = np.zeros(centers.shape) # to store old centers
centers_new = deepcopy(centers) # Store new centers

data.shape
clusters = np.zeros(n)
distances = np.zeros((n,k))

error = np.linalg.norm(centers_new - centers_old)
upper_error = error + 1

# When, after an update, the estimate of that center stays the same, exit loop
while (error + 0.02) < upper_error:
    # Measure the distance to every center
    
    distances = np.array(list(map(lambda x: point_centroid_distances(x, centers), data)))

    # Assign all training data to closest center
    clusters = np.argmin(distances, axis = 1)
    
    centers_old = deepcopy(centers_new)
    # Calculate mean for every cluster and update the center
    for i in range(k):
        centers_new[i] = np.mean(data[clusters == i], axis=0)
    upper_error = deepcopy(error)
    error = np.linalg.norm(centers_new - centers_old)
    if error < 0.02:
        break
centers_new


# calculate k using python, with the elbow method
inertia = []

# define our possible k values
possible_K_values = [i for i in range(2,10)]

# we start with 2

# iterate through each of our values
for each_value in possible_K_values:
    
    # iterate through, taking each value from 
    model = KMeans(n_clusters=each_value, init='k-means++',random_state=32)
    
    # fit it
    model.fit(df_scaled)
    
    # append the inertia to our array
    inertia.append(model.inertia_)

plt.plot(possible_K_values, inertia)
plt.title('The Elbow Method')

plt.xlabel('Number of Clusters')

plt.ylabel('Inertia')

plt.show()

# new model
model = KMeans(n_clusters=2, init='k-means++',random_state=22)

# re-fit our model
model.fit(df_scaled)

# compute an average silhouette score for each point
silhouette_score_average = silhouette_score(df_scaled, model.predict(df_scaled))

# lets see what that score it
print(silhouette_score_average)


# while that's nice, what does that tell us? there could still be a points with a negative value

# let's see the points
silhouette_score_individual = silhouette_samples(df_scaled, model.predict(df_scaled))


# iterate through to find any negative values
for each_value in silhouette_score_individual:
    if each_value < 0:
        print(f'We have found a negative silhouette score: {each_value}')
        

# Split dataset into training and testing datasets (Classical Method)
from sklearn.model_selection import train_test_split #Split dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(features, diagnosis, test_size=0.25, random_state=42)

# Reindex 
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Convert Pandas DataFrame to Numpy ndarray
X_train_values = X_train.values
y_train_values = y_train.values
X_test_values  = X_test.values
y_test_values  = y_test.values

X_train_values.shape, y_train_values.shape, X_test_values.shape, y_test_values.shape

from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans

km = KMeans(n_clusters = 2)
km

X_train_processed = km.fit_transform(X_train_values)
X_test_processed  = km.fit_transform(X_test_values)

y = km.fit_predict(X_train_processed, y_train)
y
score_k = km.score(X_test_processed, y_test)
print("score = ", score_k)

feature_importances_map = {'importance' : rfc.feature_importances_}
feature_importances_df = pd.DataFrame(feature_importances_map)
feature_importances_df.index = X_train.columns
feature_importances_df.sort_values('importance', ascending=True, inplace=True)

pip install lime

from lime import lime_text

from sklearn.pipeline import make_pipeline
from lime.lime_tabular import LimeTabularExplainer

scale = Scale()
sqrt  = Sqrt()

machine_learning_pipeline = make_pipeline(scale, sqrt, km)
class_names = ['Benign', 'Malignant']
explainer = LimeTabularExplainer(feature_names=feature_names, 
                                 class_names=class_names, 
                                 training_data=X_train_values) 

def plot(exp, label=1):
    '''
    label 0 : explaining Benign
    label 1 : explaining Malignant
    '''
    exp_list = exp.as_list()
    fig = plt.figure()
    vals = [x[1] for x in exp_list]
    #print('vals:', vals)
    names = [x[0] for x in exp_list]
    #print('names:', names)
    vals.reverse()
    names.reverse()
    colors = ['green' if x <= 0 else 'red' for x in vals]
    pos = np.arange(len(exp_list)) + .5
    plt.barh(pos, vals, align='center', color=colors)
    plt.yticks(pos, names)
    if exp.mode == "classification":
        title = 'Local explanation for class {}'.format(exp.class_names[label])
    else:
        title = 'Local explanation'
    plt.title(title)
        
    return fig

def explain(feature_vector, machine_learning_pipeline, label=1): 
    '''
    label 0 : explaining Benign
    label 1 : explaining Malignant
    '''
    exp = explainer.explain_instance(feature_vector, machine_learning_pipeline.predict_proba, num_features=9)
    fig = plot(exp, label) # explaining prediction as pyplot figure
    exp.show_in_notebook(show_table=True, show_all=False) # explaining prediction in notebook format
sample_M = raw_data.loc[5, feature_names]
type(sample_M), sample_M
p = machine_learning_pipeline.predict_proba(sample_M)
p
sample_M
sample_M_processed = data_preprocessing_pipeline.fit_transform(sample_M.values.reshape(1, -1))
sample_M_processed
sample_B_1 = raw_data.loc[4, feature_names]
explain(sample_B_1, machine_learning_pipeline, label=0)
sample_B_1 = raw_data.loc[4, feature_names]
explain(sample_B_1, machine_learning_pipeline, label=1)